# Eigene Facebook-Daten explorieren

## Vorbereitung

Mit diesem Notebook können Sie einen Teil Ihrer eigenen Facebook-Daten explorieren. Hierfür müssen Sie eine [Kopie Ihrer persönlichen Facebook-Daten herunterladen](https://www.facebook.com/help/1701730696756992). Wir arbeiten in diesem Notebook mit den Daten im `JSON`-Format, in welchem Sie Ihre Daten dementsprechend exportieren müssen, wenn Sie diese mit diesem Notebook explorieren möchten. Nachdem Sie Ihr Archiv gespeichert haben, müssen Sie die `.zip`-Datei zunächst entpacken.

**Hinweis**: Wenn Sie Ihre Facebook-Daten im `HTML`-Format exportieren, können Sie diese auch lokal in Ihrem Browser explorieren. Hierzu müssen Sie, nachdem Sie die Dateien entpackt haben, auf Ihrem Rechner lokal die Datei `index.html` öffnen. Im Netz gibt es zahlreiche Tutorials dazu, wie man seine Facebook-Daten exportiert und die `HTML`-Dateien mithilfe des Browsers explorieren kann (z.B. [dieses hier](https://www.makeuseof.com/tag/download-entire-facebook-history-data-downloader/)).

Bevor wir mit den Daten in den exportierten `JSON`-Files arbeiten können, müssen diese zunächst "repariert" werden. Grund hierfür ist ein Fehler in der [Zeichenkodierung](https://de.wikipedia.org/wiki/Zeichenkodierung), die dazu führt, dass die Textdaten nicht richtig dargestellt werden (es handelt sich hierbei um ein sogenanntes [Mojibake](https://en.wikipedia.org/wiki/Mojibake)). Um dies zu beheben, können wir den [`Python`](https://www.python.org/)-Code im Notebok [fix_fb_data_encoding](./fix_fb_data_encoding.ipynb) nutzen. Hierfür müssen Sie die entsprechenden Dateien hier hochladen.

In diesem Notebook explorieren wir Daten zu Posts, Freunden sowie Reaktionen auf Facebook (um Nachrichten aus dem Facebok Messenger zu explorieren, können Sie z.B. das Tool [FB Message Explorer](https://github.com/adurivault/FBMessage) nutzen). Hierzu benötigen folgende Dateien: `your_posts_1.json` (Facebook-Posts), `posts_and_comments.json` (Kommentare und Reaktionen) und `friends.json` (Daten zu Facebook-Freunden). Die Dateien sollten hier im Ordner *data* gespeichert werden. Öffnen Sie diesen (durch Doppelklick auf den Ordnernamen) im File Explorer auf der linken Seite und nutzen dann den *Upload Files*-Button im Menü oben links (das Symbol ist ein Pfeil über einem Strich). Wählen Sie darüber die entsprechenden `JSON`-Dateien von Ihrem lokalen Rechner aus und laden Sie diese in den Ordner *data* hoch.

**Hinweis**: Wenn Sie Facebook schon sehr lange und/oder sehr intensiv nutzen können die Dateien recht groß sein. In diesem Fall können der Upload sowie das Einlesen (im Code weiter unten) etwas länger dauern (dies gilt ggf. auch für das lokale Öffnen und Bearbeiten der Datei).

**NB**: Bevor Sie mit diesem Notebook arbeiten können, müssen Sie nach dem Upload der benötigten Dateien den Code im Notebok [fix_fb_data_encoding](./fix_fb_data_encoding.ipynb) ausführen. Wenn Sie dies gemacht haben, können Sie mit diesem Notebook fortfahren.

**Zur Erinnerung**: Ihre persönliche Kopie des Notebooks sowie alle Dateien, die Sie hochladen, werden am Ende der Nutzungssitzung gelöscht. Wenn Sie aber ganz "auf Nummer sicher gehen" wollen, können Sie die Dateien mit Ihren Facebook-Daten über den File Explorer auf der linken Seite nach dem Durcharbeiten dieses Notebooks auch manuell löschen: Rechtsklick auf den Dateinamen und dann *Delete* auswählen.

## Pakete laden

Wie auch in den anderen Notebooks müssen wir zunächst die benötigten `R`-Pakete laden. Im Vergleich zum [Notebook für die Twitterdaten](./tweets.ipynb) laden wir hier zwei weitere Pakete. Eins zur Aufbereitung der Daten ([`tidyr`](https://tidyr.tidyverse.org/)) und eins zur automatischen Erkennung von Sprache in Texten ([`cld3`](https://github.com/ropensci/cld3)). Bevor wir die Pakete laden können, müssen wir noch das Paket `stopwords` (nach-)installieren, um später im Notebook auch mit deutschsprachigen Tweets arbeiten zu können (NB: die Installation dieses Pakets kann ein paar Minuten dauern).

In [ ]:
install.packages('stopwords')

In [ ]:
library(jsonlite)
library(magrittr)
library(tidyr)
library(dplyr)
library(lubridate)
library(ggplot2)
library(scales)
library(stringr)
library(cld3)
library(tidytext)
library(stopwords)
library(wordcloud)

## Freunde

Zunächst schauen wir uns die Daten zu Freundschaften auf Facebook an. Hierzu müssen wir die entsprechende `JSON`-Datei einlesen und diese in ein `data.frame`-Objekt umwandeln, mit dem wir in `R` arbeiten können

In [ ]:
fb_friends <- fromJSON("./data/fb_friends.json")
fb_friends_df <- as.data.frame(fb_friends)

Um einen ersten Eindruck von diesen Daten zu bekommen, können wir uns die Variablennamen sowie die ersten zehn Fälle bzw. Zeilen anschauen.

In [ ]:
names(fb_friends_df)
head(fb_friends_df)

Wir wollen nun visualisieren, wie viele neue Facebook-Freunde wir pro Monat gewonnen haben. Hierzu müssen wir das Format der Zeitstempel anpassen.

In [ ]:
fb_friends_df <- fb_friends_df %>% 
  mutate(timedate = as_datetime(friends.timestamp),
         timedate = with_tz(timedate, tzone = "Europe/Berlin"))

Nun können wir uns anschauen, wie viele neue Facebook-Freunde wir im Laufe unserer Nutzung gewonnen haben.

In [ ]:
fb_friends_df %>%
  mutate(time_floor = floor_date(timedate, unit = "1 month")) %>%
  count(time_floor) %>%
  ggplot(aes(x = as.Date(time_floor), y = n)) +
  geom_bar(stat = "identity") +
  scale_x_date(breaks = function(x) seq.Date(from = min(x), to = max(x), by = "6 months"),
               minor_breaks = function(x) seq.Date(from = min(x), to = max(x), by = "1 month"),
               labels = date_format("%m-%Y"),
               expand = expansion(mult=c(0,0))) +
  scale_y_continuous(expand = expansion(mult=c(0,0.05))) +
  labs(title = "Anzahl neuer Facebook-Freunde pro Monat",
       x = "Monat",
       y = "Neue Facebook-Freunde") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

Vielleicht interessiert uns auch, wann wie unseren ersten Facebook-Freund bzw. unsere erste Facebook-Freundin hinzugefügt haben und wann wir dies zuletzt gemacht haben (vor Export der Daten).

In [ ]:
min(fb_friends_df$timedate)
max(fb_friends_df$timedate)

## Reaktionen

In diesem Abschnitt befassen wir uns mit Reaktionen auf Posts, Kommentare etc. Auch hierfür müssen wir im ersten Schritt die Daten einlesen und diese in ein Format bringen, mit dem wir arbeiten können.

In [ ]:
fb_reactions <- fromJSON("./data/fb_reactions.json")
fb_reactions_df <- as.data.frame(fb_reactions)
fb_reactions_df <- unnest(fb_reactions_df, reactions.data)

Das Format, in dem die Daten vorliegen, ist in diesem Fall etwas komplizierter (man könnte auch sagen verschachtelter), weswegen zusätzliche Aufbereitungsschritte nötig sind,

In [ ]:
reactions_df <- bind_rows(fb_reactions_df$reaction)

fb_reactions_df <- fb_reactions_df %>% 
  select(reactions.timestamp, reactions.title) %>% 
  mutate(timedate = as_datetime(reactions.timestamp),
         timedate = with_tz(timedate, tzone = "Europe/Berlin")) %>% 
  bind_cols(reactions_df)

Auch für diese Daten können wir uns anschauen, wie sie nun strukturiert sind.

In [ ]:
names(fb_reactions_df)
head(fb_reactions_df)

Was sind meine häufigsten Reaktionen?

In [ ]:
fb_reactions_df %>% 
  count(reaction) %>% 
  arrange(-n)

Auf welche Inhalte reagiere ich am häufigsten? Um diese Frage zu beantworten müssen wir aus der Variable `reactions.title` die Information extrahieren, auf was reagiert wird.

In [ ]:
fb_reactions_df <- fb_reactions_df %>% 
  mutate(react_content = factor(case_when(
    str_detect(reactions.title, "Beitrag") ~ "post",
    str_detect(reactions.title, "Foto") ~ "photo",
    str_detect(reactions.title, "Kommentar") ~ "comment",
    str_detect(reactions.title, "Video") ~ "video",
    str_detect(reactions.title, "GIF") ~ "GIF",
    str_detect(reactions.title, "Link") ~ "link",
    str_detect(reactions.title, "Album") ~ "photo album",
    str_detect(reactions.title, "Lebensereignis") ~ "life event",
    str_detect(reactions.title, "Aktivität") ~ "activity",
    str_detect(reactions.title, "Notiz") ~ "note",
    str_detect(reactions.title, "dass") ~ "like"
  )))

fb_reactions_df %>% 
  count(react_content) %>% 
  arrange(-n)

## Posts

Abschließend explorieren wir die Inhalte unserer Posts auf Facebook.

Wie immer müssen die Daten zunächst einmal eingelesen und aufbereitet werden,

In [ ]:
fb_posts_json <- fromJSON("./data/fb_posts.json")

fb_posts_df <- fb_posts_json %>% 
  unnest(data) %>% 
  select(timestamp, title, post, tags) %>% 
  mutate(timedate = as_datetime(timestamp),
         timedate = with_tz(timedate, tzone = "Europe/Berlin"))

Wie sehen die Daten aus?

In [ ]:
names(fb_posts_df)
head(fb_posts_df)

Über die Variable `title` lässt sich erkennen, um welche Art von Post es sich handelt (z.B. Post in der Chronik einer anderen Person, Status-Update, teilen von Inhalten etc.). Diese könnte man auch verwenden, um die Exploration der Daten auf bestimmte Arten von Posts zu beschränken.

Von wann sind der älteste und aktuellste Post in den Daten?

In [ ]:
min(fb_posts_df$timedate)
max(fb_posts_df$timedate)

### Mentions

Um herauszufinden, wen ich in meinen Posts am häufigsten markiere, ist weitere Datenaufbereitungsarbeit nötig.

In [ ]:
fb_posts_df <- fb_posts_df %>% 
  mutate(mentions = sapply(tags, toString))

fb_posts_df <- fb_posts_df %>% 
  mutate(nr_mentions = ifelse(mentions == "", 0, (str_count(mentions, ",")) + 1))

max(fb_posts_df$nr_mentions)

Im letzten Schritt in der obigen Code-Zelle wird die maximale Anzahl von Markierungen in einem Post festgestellt. Diese fällt tendenziell bei jedem anders aus. Daher müssen Sie den Code in der nachfolgenden Zelle ggf. entsprechend anpassen, um die für ihre Daten korrekte Anzahl von Variablen für die einzelnen Mentions festzulegen.

In [ ]:
tags <- fb_posts_df %>% 
  separate(mentions, c("tag1", "tag2", "tag3", "tag4", "tag5", "tag6", "tag7", "tag8"), ", ") %>% 
  select(tag1:tag8) %>% 
  filter(tag1 != "")

Nun können wir zählen, wenn wir in unseren Posts am häufigsten markieren?

In [ ]:
tags %>% 
  pivot_longer(everything(), names_to = "tag_nr", values_to = "name") %>% 
  filter(!is.na(name)) %>% 
  count(name) %>% 
  arrange(-n) %>% 
  head(10)

### Worthäufigkeiten

Das meiste, was in diesem Abschnitt gemacht wird, ist analog zum dem, was im Notebook [tweets.ipynb](./tweets.ipynb) für Tweets geschieht. Da die Daten unterschiedlich strukturiert sind, unterscheidet sich der Code an einigen Stellen allerdings.

Für unsere Analyse möchten wir Links (URLs) und Mentions ausschließen. Daher legen wir mithilfe von [Regular Expressions](https://de.wikipedia.org/wiki/Regul%C3%A4rer_Ausdruck) Muster fest, um diese entsprechen aus den Texten der Posts zu entfernen.

In [ ]:
url_pattern <- "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
tag_pattern <- "@\\[.*\\]"

fb_posts_df <- fb_posts_df %>% 
  mutate(post_clean = str_replace(post, url_pattern, ""),
         post_clean = str_replace(post_clean, tag_pattern, ""))

Anders als bei den Daten zu den Tweets ist in denjenigen zu den Facebook-Posts die Information dazu, in welcher Sprache diese verfasst wurden (bzw. welche Sprache das genutzte automatisierte Verfahren erkannt hat) nicht enthalten. Daher müssen wir für die Erkennung der Sprache ein zusätzliches Paket nutzen. Wir verwenden das Paket [`cld3`](https://github.com/ropensci/cld3), welches eine Implementation des *Compact Language Detector 3* von Google für `R` ist. Ähnlich wie bei der Language Detection von Twitter funktioniert das natürlich nicht 100%ig. Es gibt auch weitere `R`-Pakete, die man zur Erkennung der Sprache von Texten nutzen kann, wie z.B. das Paket [`tecxtcat`](https://cran.r-project.org/web/packages/textcat/index.html). Man kann diese Pakete auch in Kombination verwenden. Welche Lösungen man nutzt bzw. kombiniert hängt auch davon ab, wie wichtig für die jeweilige Aufgabe [False Positives und False Negatives](https://en.wikipedia.org/wiki/False_positives_and_false_negatives) bei der Erkennung sind.

In [ ]:
fb_posts_df <- fb_posts_df %>% 
  mutate(lang_g = detect_language(fb_posts_df$post_clean))

Welche Sprachen wurden erkannt und wie häufig jeweils?

In [ ]:
fb_posts_df %>% 
  count(lang_g) %>% 
  arrange(-n) %>% 
  head(10)

Die nächsten Schritte sind dieselben, wie für die Auszählung und Visualisierung der Worthäufigkeiten in Tweets, wie sie im Notebook [tweets.ipynb](./tweets.ipynb) umgesetzt sind: Die Tweets werden in einzelne Wörter aufgeteilt ([Tokenization](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)) und dann werden [Stoppwörter](https://de.wikipedia.org/wiki/Stoppwort) entfernt. An den Beispielen wird deutlich, dass es auf die Ergebnisse einen großen Einfluss hat, welche der sogenannten Preprocessing-Methoden (z.B. Language Detection, Tokenization und Stopword Removal) man einsetzt, in welcher Reihenfolge man dies macht und welche Tools man dazu verwendet.

Zuerst schauen wir uns die gesamten Posts an (d.h. nicht nach erkannter Sprache gefiltert).

In [ ]:
fb_posts <- fb_posts_df %>% 
  unnest_tokens(word, post_clean)

stop_words_de <- get_stopwords(language = "de")

fb_posts_clean <- fb_posts %>% 
  filter(!word %in% stop_words$word,
         !word %in% stop_words_de$word,
         str_detect(word, "[a-z]"))

In [ ]:
fb_posts_clean %>% 
  count(word, sort = T) %>% 
  top_n(10) %>% 
  mutate(word = reorder(word, n)) %>% 
  ggplot(aes(x = word, y = n)) +
  geom_col() +
  labs(x = NULL, y = "Häufigkeit") +
  coord_flip()

Nun vergleichen wir die deutsch- und englischsprachigen Facebook-Posts.

In [ ]:
posts_de <- fb_posts %>% 
  filter(lang_g == "de",
         !word %in% stop_words_de$word,
         str_detect(word, "[a-z]"))

posts_de %>% 
  count(word, sort = T) %>% 
  top_n(15)

In der Liste der häufigsten deutschen Wörter in den Posts sehen wir einige Wörter, die man tendenziell auch als Stoppwörter bezeichnen kann. Diese sind offenbar nicht in der Liste der deutschen Stoppwörter aus dem Paket `stopwords` enthalten. Um diese aus unseren Analysen auszuschließen, können wir zusätlich eine Liste mit eigenen Stoppwörtern definieren (im nachfolgenden Code `custom_stops` genannt). Für Ihre eigenen Daten müssen Sie diese Liste natürlich entsprechend anpassen.

In [ ]:
custom_stops <- c("dass", "gibt", "mal", "ja")

posts_de %>% 
  filter(!word %in% custom_stops) %>% 
  count(word, sort = T) %>% 
  top_n(10) %>% 
  mutate(word = reorder(word, n)) %>% 
  ggplot(aes(x = word, y = n)) +
  geom_col() +
  labs(x = NULL, y = "Häufigkeit") +
  coord_flip()

Zum Vergleich die Wörter aus den Posts, die als englischsprachig identifiziert wurden.

In [ ]:
posts_en <- fb_posts %>% 
  filter(lang_g == "en",
         !word %in% stop_words$word,
         str_detect(word, "[a-z]"))

posts_en %>% 
  count(word, sort = T) %>% 
  top_n(10) %>% 
  mutate(word = reorder(word, n)) %>% 
  ggplot(aes(x = word, y = n)) +
  geom_col() +
  labs(x = NULL, y = "Häufigkeit") +
  coord_flip()

Und der direkte visuelle Vergleich der Worthäufigkeiten zwischen deutschen und englischen Posts (hierfür müssen wir die Daten wieder kombinieren).

In [ ]:
posts_combined <- posts_en %>% 
  bind_rows(posts_de)

posts_combined %>% 
  filter(!word %in% custom_stops) %>% 
  count(word, lang_g, sort = T) %>% 
  ungroup %>% 
  group_by(lang_g) %>%
  top_n(10, n) %>%
  ungroup() %>%
  mutate(word = reorder(word, n, fill = lang_g)) %>% 
  ggplot(aes(x = word, y = n)) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~lang_g, scales = "free_y") +
  labs(x = NULL, y = "Häufigkeit") +
  coord_flip()

Und zum Abschluss nochmal eine bunte Wortwolke mit Wörtern aus allen Posts. Ggf. macht es hier für Ihre eigenen Daten Sinn, die Mindeshäufigkeit (`min.freq`) sowie die maximale Anzahl (`max.words`) der dargestellten Wörter anzupassen.

In [ ]:
word_freqs <- fb_posts_clean %>% 
  count(word) %>% 
  rename(freq = n)

wordcloud(words = word_freqs$word,
          freq = word_freqs$freq,
          min.freq = 20,
          max.words = 50,
          random.order = FALSE,
          rot.per = 0.35,
          colors = brewer.pal(8, "Dark2"))